In [1]:
from math import pi
import matplotlib.pyplot as plt
from pennylane import numpy as np
import pennylane as qml
from pennylane.optimize import AdamOptimizer
import datetime
now=datetime.datetime.now

In [2]:
num_qubits=2
dev=qml.device('default.qubit', wires=num_qubits)
seed = 40
np.random.seed(seed)

In [3]:
t=np.linspace(-10, 10, 40)

In [4]:
@qml.qnode(dev, diff_method="backprop", interface="autograd")
def hepler(encoded_x,theta):
    qml.QubitStateVector(encoded_x,wires=[i for i in range(num_qubits)])
    # qml.RY(wires=0,phi=theta[0])
    # qml.RX(wires=0,phi=theta[1])
    qml.StronglyEntanglingLayers(weights=theta, wires=range(num_qubits))
    return qml.probs(wires=[0,1])

In [5]:
def encoder(x):
    vector_x = [1,x,x**2,x**3]
    # vector_x = [1,x]
    norm_x = np.linalg.norm(vector_x)
    return vector_x/norm_x , norm_x

In [6]:

def f(x,theta,w):
    encoded_x,norm_x = encoder(x)
    f_val = hepler(encoded_x,theta)
    # rescale_coef = abs(w[0])
    rescale_coef = w[0]
    return (f_val[0]*(norm_x**2))*rescale_coef+w[1]
    # print(qml.draw(hepler)(encoded_x))

In [7]:
# f(-1.7)

In [8]:
def f_test(t):
    return -5*t**2-20*t-10

In [9]:
def callbackF(parameters):
    # global current_loss

    print("Current loss:", cost(parameters)) 


def cost(params):
    global theta
    global w
    # Reshape the flattened theta back to its original shape
    theta_shape = theta.shape
    w_shape = w.shape
    w, theta_flat = params[:2], params[2:]
    theta = theta_flat.reshape(theta_shape)
   
    loss = 0
    for x in t:
        loss += (f(x,theta,w)-f_test(x))**2/len(t)
    
    # print(w)
    # print(theta)
    # raise KeyError
    return loss

In [10]:
import numpy as np
from scipy.optimize import minimize

num_layers = 2

# theta = np.random.uniform(0, 2 * np.pi, size=(2,2))

shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
print(shape)
theta = np.random.random(size=shape)

w = np.zeros(2)

# initial_params = np.concatenate([w, theta.flatten()])  # Flatten theta to a 1D array
initial_params = np.concatenate([w, theta.flatten()])  # Flatten theta to a 1D array

result = minimize(cost, initial_params, callback=callbackF, method='BFGS', options={'gtol': 1E-2})
# Extract the optimized parameters
w, theta_flat = result.x[:2], result.x[2:]
theta = theta_flat.reshape(theta.shape)

(2, 2, 3)
Current loss: 16207.922093929765
Current loss: 16197.725520419326
Current loss: 16179.246043277552
Current loss: 15994.51514444645
Current loss: 15594.451115692971
Current loss: 15492.79247467659
Current loss: 15360.8612558235
Current loss: 14851.00433832831
Current loss: 14637.831405885438
Current loss: 14391.777462560498
Current loss: 14230.13509143207
Current loss: 12731.668330296781
Current loss: 10296.315826614189
Current loss: 9531.755985110385
Current loss: 8705.099170463716
Current loss: 8151.27772614058
Current loss: 8109.409700284486
Current loss: 8069.41115475008
Current loss: 8009.663842247239
Current loss: 7923.305036481716
Current loss: 7856.131583623562
Current loss: 7809.549136451748
Current loss: 7744.974159011244
Current loss: 7628.797660002491
Current loss: 7442.531512017625
Current loss: 7306.659097167198
Current loss: 7237.633053729939
Current loss: 7156.096500486868
Current loss: 7027.710264075367
Current loss: 6972.401526762313
Current loss: 6884.110149

In [11]:
print(w)

[-24.84383684   9.88788625]


In [12]:

# Calculate corresponding y values for both functions
y1 = [f(x,theta[0],w) for x in t]
y2 = [f_test(x) for x in t]


# Create the plot
plt.figure(figsize=(10, 6))  # Set the figure size
plt.scatter(t, y1, label='Quantum f(x)', linewidth=2, color='b')  # Plot the first function
plt.plot(t, y2, label='Analytic f(x)', linewidth=2, color='r')  # Plot the second function
plt.xlabel('x')  # X-axis label
plt.ylabel('y')  # Y-axis label
# plt.title('Plot of Two Functions')  # Title of the plot
plt.grid(True, linestyle='--', alpha=0.6)  # Add a grid
plt.legend()  # Show the legend
plt.axhline(0, color='black', linewidth=0.5)  # Add horizontal line at y=0
plt.axvline(0, color='black', linewidth=0.5)  # Add vertical line at x=0
# plt.ylim([-5, 10])  # Set y-axis limits


ValueError: Weights tensor must have second dimension of length 2; got 3